# Pilot Experiments on Distributed AI Dynamics



In [1]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings.gpt4all import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All
from langchain.chains import RetrievalQA
from collections import Counter
import numpy as np
from collections import defaultdict
import json
from pathlib import Path
from langchain.prompts import PromptTemplate

ModuleNotFoundError: No module named 'langchain'

In [2]:
# file_path='./combined-newsqa-data-v1-small.json'
file_path='/Users/wk77/Library/CloudStorage/OneDrive-DrexelUniversity/Documents/data/combined-newsqa-data-v1.json'
data = json.loads(Path(file_path).read_text())

In [ ]:
# data

In [ ]:
# for story in data['data']:
#     print(story['text'])
#     print("\n--- End of story ---\n")

In [3]:
# Helper function to calculate Exact Match (EM) score
def calculate_em(predicted, actual):
    return int(predicted == actual)

# Function to calculate the token-wise F1 score for text answers
def calculate_token_f1(predicted, actual):
    predicted_tokens = predicted.split()
    actual_tokens = actual.split()
    common_tokens = Counter(predicted_tokens) & Counter(actual_tokens)
    num_same = sum(common_tokens.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(predicted_tokens)
    recall = 1.0 * num_same / len(actual_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

# Helper function to extract answer ranges from the consensus field
def extract_ranges(consensus):
    if 's' in consensus and 'e' in consensus:
        return [(consensus['s'], consensus['e'])]
    return []


template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Also provide me the source for your answer. Explain how to get the answer step by step.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [4]:
import time
import logging
# logging.basicConfig(level=logging.WARNING)  # This will show only warnings and errors
logging.basicConfig(level=logging.ERROR)

# Parameters
max_stories = 100
# Define chunk sizes and overlap percentages
# chunk_sizes = [200,400,800,1600]
# overlap_percentages = [0,0.1,0.2,0.3]  # Expressed as percentages (0.1 = 10%)
chunk_sizes = [125,150,175,200]
overlap_percentages = [0,0.2,0.4]  # Expressed as percentages (0.1 = 10%)

# Results storage
f1_results = defaultdict(list)
em_results = defaultdict(list)
text_results = []

# Initialize the language model and the QA chain
# llm = GPT4All(model="C:/Users/24075/AppData/Local/nomic.ai/GPT4All/ggml-model-gpt4all-falcon-q4_0.bin", max_tokens=2048)
llm = GPT4All(model="/Users/wk77/Library/CloudStorage/OneDrive-DrexelUniversity/Documents/data/gpt4all/models/gpt4all-falcon-q4_0.gguf", max_tokens=2048)

# The following code would iterate over the stories and questions to calculate the scores
start_time = time.time()
print(f"{start_time} Started.")

output_file_path = "results/scores_20231116.txt"

with open(output_file_path, 'w') as file:
    # instantiate embedding ONCE
    word_embed = GPT4AllEmbeddings()
    for chunk_size in chunk_sizes:
        print(f"\n{time.time()-start_time} Processing chunk size {chunk_size}:")
        last_time = time.time()
        for overlap_percentage in overlap_percentages:
            actual_overlap = int(chunk_size * overlap_percentage)
            print(f"\n{time.time()-start_time}\t{time.time()-last_time}\tOverlap [{overlap_percentage}] {actual_overlap}")
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=actual_overlap)
    
            i = 0
            for story in data['data']:
                i+=1
                if i>max_stories:
                    break
                now_time = time.time()
                print(f"\n{now_time-start_time}\t{now_time-last_time}\t\tstory {i}: ",end='')
                last_time = now_time
    
                all_splits = text_splitter.split_text(story['text'])
                # print(f"[after split]")
                vectorstore = Chroma.from_texts(texts=all_splits, embedding=word_embed)
                # print(f"[after vector store]")
                qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever(), return_source_documents=True)
                # print(f"[after qa chain]")
    
                j = 0
                for question_data in story['questions']:
                    j+=1
                    # print(f"{time.time()-start_time}\t\t\tquestion {j}")
                    print('.', end='')
    
                    # TODO: embed query and perform similarity_search_by_vector() instead
                    question = question_data['q']
                    question_vector = word_embed.embed_query(question)
                    # docs = vectorstore.similarity_search(question)
                    docs = vectorstore.similarity_search_by_vector(question_vector)
                    answer_ranges = extract_ranges(question_data['consensus'])
                    
                    # Get the prediction from the model
                    result = qa_chain({"query": question})
                    
                    # Check if the predicted answer is in the expected format (string)
                    predicted_answer = result['result']
                    if isinstance(predicted_answer, dict):
                        # If it's a dictionary, you need to adapt this part of the code to extract the answer string
                        predicted_answer = predicted_answer.get('answer', '')  # Assuming 'answer' is the key for the answer string
                    elif not isinstance(predicted_answer, str):
                        # If the answer is not a string and not a dictionary, log an error or handle it appropriately
                        print(f"Unexpected format for predicted answer: {predicted_answer}")
                        continue  # Skip to the next question
                    actual_answer = story['text'][answer_ranges[0][0]:answer_ranges[0][1]] if answer_ranges else ""
                    
                    # If there is an actual answer, get it from the story text using the character ranges
                    if answer_ranges:
                        actual_answer = story['text'][answer_ranges[0][0]:answer_ranges[0][1]]
                    else:
                        actual_answer = ""
                    
                    # Calculate the scores
                    em_score = calculate_em(predicted_answer, actual_answer)
                    f1_score_value = calculate_token_f1(predicted_answer, actual_answer)
                    file.write(f"{chunk_size}\t{overlap_percentage}\t{i}\t{j}\t{f1_score_value:.4f}\t{em_score:.2f}\n")
    
                    # Store the scores
                    em_results[(chunk_size, overlap_percentage)].append(em_score)
                    f1_results[(chunk_size, overlap_percentage)].append(f1_score_value)
    
                # delete object for memory
                del qa_chain
                del vectorstore
                del all_splits
            # delete splitter instance
            del text_splitter

1700098060.115932 Started.

0.4852561950683594 Processing chunk size 125:bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


0.48534297943115234	2.86102294921875e-06	Overlap [0] 0

0.48535704612731934	1.5974044799804688e-05		story 1: .........
20.461195945739746	19.975838899612427		story 2: .........
39.16641592979431	18.705219984054565		story 3: .........
57.53596782684326	18.36955189704895		story 4: ........
72.52744317054749	14.991475343704224		story 5: .........
90.23438715934753	17.70694398880005		story 6: .........
109.65649509429932	19.422107934951782		story 7: .........
125.18215203285217	15.525656938552856		story 8: ............
149.8979618549347	24.71580982208252		story 9: ............
173.89833402633667	

INFO:backoff:Backing off send_request(...) for 0.6s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


........
7356.833469152451	19.862926244735718		story 31: .........
7375.965399980545	19.131930828094482		story 32: ...........
8363.264006137848	987.2986061573029		story 33: .

INFO:backoff:Backing off send_request(...) for 0.8s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


...........
8386.737882137299	23.473875999450684		story 34: ........
8401.1184091568	14.380527019500732		story 35: .........
8418.827260017395	17.708850860595703		story 36: .........
8439.293065786362	20.465805768966675		story 37: ............................
9398.619193077087	959.3261272907257		story 38: .

...

........
44578.622425079346	22.67533302307129		story 52: .........
44594.38988995552	15.767464876174927		story 53: .........
44609.568408966064	15.178519010543823		story 54: ............
44720.35321807861	110.78480911254883		story 55: .........
44738.17446398735	17.821245908737183		story 56: ........
44751.80032706261	13.625863075256348		story 57: .........
44767.08034801483	15.280020952224731		story 58: .........
44785.02545595169	17.94510793685913		story 59: ............
44806.4699921608	21.444536209106445		story 60: .........
44824.52652192116	18.056529760360718		story 61: .........
44846.88792204857	22.36140012741089		story 62: .........
44866.30271100998	19.414788961410522		story 63: .........
44883.480097055435	17.177386045455933		story 64: ...........
44908.85395908356	25.37386202812195		story 65: .........
44925.066494226456	16.21253514289856		story 66: .........
44943.65586400032	18.589369773864746		story 67: .........
44959.319945812225	15.664081811904907		story 68: .........

INFO:backoff:Backing off send_request(...) for 0.1s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


...........
57174.8662340641	23.31519317626953		story 34: ........
57190.20205616951	15.335822105407715		story 35: .........
57212.804349184036	22.602293014526367		story 36: .........
57232.27476501465	19.470415830612183		story 37: ............................
57286.49087810516	54.21611309051514		story 38: .........
57303.94404697418	17.453168869018555		story 39: .........
57323.123461961746	19.179414987564087		story 40: .........
57337.802693128586	14.6792311668396		story 41: .........
57355.2728869915	17.47019386291504		story 42: ........
57370.54309487343	15.27020788192749		story 43: ............
57396.94052815437	26.397433280944824		story 44: .........
57416.90551996231	19.964991807937622		story 45: ........
57431.92878293991	15.023262977600098		story 46: .........
57448.9106400013	16.98185706138611		story 47: .........
57466.88275098801	17.972110986709595		story 48: ............
57489.211597919464	22.32884693145752		story 49: ........
57504.91726899147	15.705671072006226		story 50

INFO:backoff:Backing off send_request(...) for 0.1s (posthog.request.APIError: [PostHog] <!DOCTYPE html>
<html lang="en">
    <head>
        <title>PostHog</title>
            <meta name="viewport" content="width=device-width, initial-scale=1">
            <meta charset="utf-8">
            <link rel="apple-touch-icon" sizes="180x180" href="/static/icons/apple-touch-icon.png?v=2021-04-29">
            <link rel="icon" type="image/png" sizes="32x32" href="/static/icons/favicon-32x32.png?v=2021-04-29">
            <link rel="icon" type="image/png" sizes="16x16" href="/static/icons/favicon-16x16.png?v=2021-04-29">
            <link rel="shortcut icon" href="/static/icons/favicon.ico?v=2021-04-29">
            <link rel="mask-icon" href="/static/icons/safari-pinned-tab.svg?v=2021-04-29" color="#FF053C">
            <link rel="manifest" href="/static/site.webmanifest?v=2021-04-29">
            <meta name="apple-mobile-web-app-title" content="PostHog">
            <meta name="application-nam


59616.918145895004	23.818562984466553		story 55: .........
59632.68617486954	15.768028974533081		story 56: ........
59646.446955919266	13.760781049728394		story 57: .........
59662.33335995674	15.886404037475586		story 58: .........
59679.96654510498	17.633185148239136		story 59: ............
59700.169140815735	20.202595710754395		story 60: .........
59717.69806098938	17.52892017364502		story 61: .........
59740.4294090271	22.731348037719727		story 62: .........
59761.816532850266	21.387123823165894		story 63: .........
59779.863512039185	18.046979188919067		story 64: ...........
59802.538803100586	22.675291061401367		story 65: .........
59818.10990190506	15.571098804473877		story 66: .........
59835.242115974426	17.132214069366455		story 67: .........
59853.02603507042	17.783919095993042		story 68: .........
59870.455245018005	17.42920994758606		story 69: ........
59888.821825027466	18.36658000946045		story 70: ........
59907.06362700462	18.241801977157593		story 71: .........
59923.

In [5]:
# Calculate the average F1 and EM scores for each configuration
for config, scores in f1_results.items():
    avg_f1 = np.mean(scores)
    avg_em = np.mean(em_results[config])
    f1_results[config] = avg_f1
    em_results[config] = avg_em
    print(f"Chunk size {config[0]} with overlap {config[1]}% - Average F1: {avg_f1:.3f}, EM: {avg_em:.2f}")

# Output the results
print(f1_results)
print(em_results)

Chunk size 125 with overlap 0% - Average F1: 0.092, EM: 0.00
Chunk size 125 with overlap 0.2% - Average F1: 0.083, EM: 0.00
Chunk size 125 with overlap 0.4% - Average F1: 0.075, EM: 0.00
Chunk size 150 with overlap 0% - Average F1: 0.071, EM: 0.00
Chunk size 150 with overlap 0.2% - Average F1: 0.069, EM: 0.00
Chunk size 150 with overlap 0.4% - Average F1: 0.065, EM: 0.00
Chunk size 175 with overlap 0% - Average F1: 0.065, EM: 0.00
Chunk size 175 with overlap 0.2% - Average F1: 0.063, EM: 0.00
Chunk size 175 with overlap 0.4% - Average F1: 0.061, EM: 0.00
Chunk size 200 with overlap 0% - Average F1: 0.063, EM: 0.00
Chunk size 200 with overlap 0.2% - Average F1: 0.064, EM: 0.00
Chunk size 200 with overlap 0.4% - Average F1: 0.065, EM: 0.00
defaultdict(<class 'list'>, {(125, 0): 0.0915107476656089, (125, 0.2): 0.08319619994489912, (125, 0.4): 0.0753000168320197, (150, 0): 0.07050237854425973, (150, 0.2): 0.06870684635328551, (150, 0.4): 0.06472554049142945, (175, 0): 0.06461816510684007, 

In [ ]:
# # Print out the F1 and EM Results
# print("\nF1 and EM Results:")
# for config in f1_results:
#     chunk_size, overlap_percentage = config
#     print(f"Chunk Size: {chunk_size}, Overlap Percentage: {overlap_percentage}%, F1: {f1_results[config]}, EM: {em_results[config]}")